<a href="https://colab.research.google.com/github/Calcifer777/learn-deep-learning/blob/main/learn-rag/bge_ft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources

- Datasets: https://github.com/kongds/LightXML

In [1]:
%%bash

pip uninstall -y transformers # needed for working in colab
pip install transformers flagembedding faiss-gpu  # or faiss-cpu, can't have both

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
  Created wheel for flagembedding: filename=FlagEmbedding-1.2.5-py3-none-any.whl size=43015 sha256=1e06015f7ce6733c3cdab2f468a63f9dc2ab1d06418c4a9db26ba80f6c08fe5b
  Stored in directory: /root/.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


# Parameters

In [308]:
model_name = "BAAI/bge-m3"

lowest_label = 54
highest_label = 80
max_neg_samples = 8

In [283]:
google_url_tmpl = " https://drive.google.com/uc?id={file_id}"

datasets = {
    "amazon13k": "1VwHAbri6y6oh8lkpZ6sSY_b1FRNnCLFL",  # alternative link, but only with label ids
    "amazon13k-full": "17rVRDarPwlMpb3l5zof9h34FlwbpTu4l",
}

## Download Data

In [364]:
import json
from itertools import compress
import random

import gdown
import numpy as np
from tqdm import tqdm

from torch.utils.data import Dataset
from FlagEmbedding  import BGEM3FlagModel, FlagModel

import faiss

In [287]:
ds_name = "amazon13k"

gdown.download(
    google_url_tmpl.format(file_id=datasets[ds_name]),
    f"./datasets/{ds_name}/",
)

Downloading...
From (original):  https://drive.google.com/uc?id=1VwHAbri6y6oh8lkpZ6sSY_b1FRNnCLFL
From (redirected): https://drive.google.com/uc?id=1VwHAbri6y6oh8lkpZ6sSY_b1FRNnCLFL&confirm=t&uuid=d2e9453b-4ae7-4b80-a56d-dd791352b95b
To: /content/datasets/amazon13k/AmazonCat-13K.tar.gz
100%|██████████| 796M/796M [00:30<00:00, 25.9MB/s]


'./datasets/amazon13k/AmazonCat-13K.tar.gz'

In [288]:
ds_name = "amazon13k-full"

gdown.download(
    google_url_tmpl.format(file_id=datasets[ds_name]),
    f"./datasets/{ds_name}/",
)

Downloading...
From (original):  https://drive.google.com/uc?id=17rVRDarPwlMpb3l5zof9h34FlwbpTu4l
From (redirected): https://drive.google.com/uc?id=17rVRDarPwlMpb3l5zof9h34FlwbpTu4l&confirm=t&uuid=f5e0687e-c5d3-4a30-8ef0-223a4930b9bb
To: /content/datasets/amazon13k-full/AmazonCat-13K.raw.zip
100%|██████████| 837M/837M [00:26<00:00, 31.2MB/s]


'./datasets/amazon13k-full/AmazonCat-13K.raw.zip'

In [289]:
%%bash

tar -xvf ./datasets/amazon13k/AmazonCat-13K.tar.gz -C ./datasets/amazon13k/

AmazonCat-13K/train_raw_texts.txt
AmazonCat-13K/test_raw_texts.txt
AmazonCat-13K/train_labels.txt
AmazonCat-13K/test_labels.txt


In [7]:
%%bash

head -5 ./datasets/amazon13k/AmazonCat-13K/train_raw_texts.txt
head -5 ./datasets/amazon13k/AmazonCat-13K/train_labels.txt

New Englanders on the Ohio Frontier: Migration and Settlement of Worthington, Ohio  /SEP/  Virginia E. McCormick served on the faculties of Pennsylvania State University, Iowa State University, and, most recently, Ohio State University. Her previous books include Farm Wife: A Self Portrait, 1886-1896 and Scioto Company Descendants: Genealogies of the Original Proprietors of Worthington, Ohio./i>Robert W. McCormick is professor emeritus and former assistant vice president for continuing education at Ohio State University. He is the author of Cockney: The Story of the 696th Armored Field Artillery Battalion in World War II.The McCormicks are coauthors of A. B. Graham: Country Schoolmaster and Extension Pioneer; Worthington Landmarks: Photo-Essays of Historic Worthington Properties; and Probing Worthington's Heritage.
Le Petit Prince  /SEP/  Deluxe Double Disc Musical Spectacular that Comes with a 52 Page Booklet with Libretto and Notes.
Nesco FS-120T American Harvest Food Slicer with Til

In [292]:
%%bash

unzip -j ./datasets/amazon13k-full/AmazonCat-13K.raw.zip "AmazonCat-13K.raw/Yf.txt" -d ./datasets/amazon13k/

Archive:  ./datasets/amazon13k-full/AmazonCat-13K.raw.zip
  inflating: ./datasets/amazon13k/labels.txt/Yf.txt  


# Filter only a subset of the dataset

In [300]:
with open("./datasets/amazon13k/Yf.txt", "r", encoding="ISO-8859-1") as fp:
    lines = [l.strip() for l in fp.readlines()]

id2labels = dict(zip(range(len(lines)), lines))

In [303]:
list(id2labels.items())[:5]

[(0, '& guitar'),
 (1, '& magic'),
 (2, '& nets'),
 (3, '& screens'),
 (4, '& sleeving')]

In [304]:
with open("./datasets/amazon13k/AmazonCat-13K/train_labels.txt", "r") as fp:
    labels = [[int(x) for x in l.split()] for l in fp.readlines()]

In [329]:
selection_mask = [l[0] >= lowest_label and l[0] <= highest_label for l in labels]

In [330]:
labels_sel = [l[0] for l in compress(labels, selection_mask)]

In [333]:
len(labels), len(labels_sel)

(1186239, 1732)

In [334]:
len(set(labels_sel))

20

In [335]:
labels = labels_sel

In [336]:
corpus = list()

with open("./datasets/amazon13k/AmazonCat-13K/train_raw_texts.txt", "r") as fp:
    corpus = [l.strip() for l in fp.readlines()]

corpus = list(compress(corpus, selection_mask))

In [337]:
assert len(labels) == len(corpus)

# Find Hard negatives

## Create corpus embeddings

https://github.com/FlagOpen/FlagEmbedding/blob/master/FlagEmbedding/baai_general_embedding/finetune/hn_mine.py

In [338]:
def create_index(embeddings, use_gpu=False):
    index = faiss.index_factory(embeddings.shape[1], "Flat")
    embeddings = np.asarray(embeddings, dtype=np.float32)
    if use_gpu:
        co = faiss.GpuMultipleClonerOptions()
        co.shard = True
        co.useFloat16 = True
        index = faiss.index_cpu_to_all_gpus(index, co=co)
    index.add(embeddings)
    return index

In [339]:
def batch_search(
    index,
    query,
    topk: int = 200,
    batch_size: int = 64,
):
    all_scores, all_inxs = [], []
    for start_index in tqdm(range(0, len(query), batch_size), desc="Batches", disable=len(query) < 256):
        batch_query = query[start_index:start_index + batch_size]
        batch_scores, batch_inxs = index.search(np.asarray(batch_query, dtype=np.float32), k=topk)
        all_scores.extend(batch_scores.tolist())
        all_inxs.extend(batch_inxs.tolist())
    return all_scores, all_inxs

In [247]:
model = FlagModel(model_name)

In [340]:
embeddings = model.encode(corpus, batch_size=256)

Inference Embeddings: 100%|██████████| 7/7 [00:49<00:00,  7.03s/it]


## Setup Index

In [341]:
faiss_index = create_index(embeddings)

In [360]:
q_scores, q_indices = batch_search(faiss_index, embeddings, topk=500)

Batches: 100%|██████████| 28/28 [00:00<00:00, 50.52it/s]


## Find negatives

In [366]:
all_negs = []

for idx, (scores, indices) in enumerate(zip(q_scores, q_indices)):
    ref_label = labels[idx]
    neg_samples = []
    for i in indices[1:]:
        q_label = labels[i]
        if q_label != ref_label:
            neg_samples.append(i)
    if len(neg_samples) > max_neg_samples:
        neg_samples = random.sample(neg_samples, max_neg_samples)
    all_negs.append(neg_samples)

In [367]:
sample_idx = 10

print("Text:", corpus[sample_idx])
print("Label id:", labels[sample_idx])
print("Label:", id2labels[labels[sample_idx]])
print("\nNegative samples")
for i in all_negs[sample_idx]:
    print("\tText:", corpus[i])
    print("\tLabel:", id2labels[labels[i]])

Text: SAIT 58167 Open Coat Aluminum Oxide 3-Inch x 21-Inch AO-X 120 Grit Sanding Belt, 10-Pack  /SEP/  United Abrasives, Inc./SAIT is a privatelyheld company that was established in 1970. Ourrelationship with SAIT, an Italian corporation foundedin 1953, began as we first manufactured coated abrasivebelts. By 1975 we had expanded our production to includebonded abrasives. The strong commitment between UnitedAbrasives, Inc. and SAIT continues today. We are well-known worldwide for quality, consistency, and the highest levels of service.  We are one of the largest and leading manufacturers of abrasiveproducts in the world. We only use premium materials. We demand consistent manufacturing operations and have the most rigorous quality control testing in the industry. United Abrasives manufactures a full line of bonded abrasives: grinding wheels, cuttingwheels, cup wheels, cones, plugs and a host of similar bonded products. We also manufacture a wide variety of sanding sheets, belts, rolls, 

In [374]:
with open("hnn.jsonl", "w") as fp:
    for i, text in enumerate(corpus):
        data = dict(
            text=text,
            neg=[corpus[neg_idx] for neg_idx in all_negs[i]],
        )
        fp.write(json.dumps(data, ensure_ascii=False)+"\n")

In [379]:
%%bash

head -1 hnn.jsonl | python -m json.tool

{
    "text": "Dignity Extra Duty Pads Extra-Double Duty 30/Bag  /SEP/  Dignity Extra Duty Pads are for moderate to heavy protection. Best used with Dignity garment or regular pants. Double pad is made of fluff filling and a soft, non-woven covering. Pad can be cut for single use, or layered for triple protection. No moisture-proof backing or adhesive strip.",
    "neg": [
        "Eagle U778-1000 - 6 inch SUPER-TACK Yellow-Film Discs - Grit P1000 - (Job-Pak) - 5 Discs/Pack - 1 Pack  /SEP/  Long lasting high performance Yellow-Film discs cut extremely fast and uniform. Designed for use in topcoat sanding to remove excess orange peel, over-runs, and dust nibs. Can also be used for light scuff sanding before painting. SUPER-TACK system provides cooler sanding and the convenience of easy on and off from disc pad. Recommended using with Cushion Pad (Part no. 04631). SUPER-TACK= Velour backing",
        "3M 81433 4-Inch x 24-Inch Purple Regalite Resin Bond 120 Grit Cloth Sanding Belt - 5 Pa

# Model

In [ ]:
faiss.StandardGpuResources

# Dataset

In [13]:
class AmazonDataset(Dataset):

    def __init__(self, path_data: str, path_labels: str):
        super().__init__()
        with open(path_data, "r") as fp:
            self.data = [l.strip() for l in fp.readlines()]
        with open(path_labels, "r") as fp:
            self.labels = [l.strip().split() for l in fp.readlines()]

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> SimilarityGroupSample:
        item = self.data[index]
        return SimilarityGroupSample(obj=item, group=self.labels[index][0])

    @property
    def num_labels(self):
        return len(set(x for l in ds_train.labels for x in l))

In [46]:
ds_train = AmazonDataset(
    path_data="./datasets/amazon13k/AmazonCat-13K/train_raw_texts.txt",
    path_labels="./datasets/amazon13k/AmazonCat-13K/train_labels.txt"
)

In [50]:
ds_valid = AmazonDataset(
    path_data="./datasets/amazon13k/AmazonCat-13K/test_raw_texts.txt",
    path_labels="./datasets/amazon13k/AmazonCat-13K/test_labels.txt"
)

In [47]:
ds_train[0]

SimilarityGroupSample(obj="New Englanders on the Ohio Frontier: Migration and Settlement of Worthington, Ohio  /SEP/  Virginia E. McCormick served on the faculties of Pennsylvania State University, Iowa State University, and, most recently, Ohio State University. Her previous books include Farm Wife: A Self Portrait, 1886-1896 and Scioto Company Descendants: Genealogies of the Original Proprietors of Worthington, Ohio./i>Robert W. McCormick is professor emeritus and former assistant vice president for continuing education at Ohio State University. He is the author of Cockney: The Story of the 696th Armored Field Artillery Battalion in World War II.The McCormicks are coauthors of A. B. Graham: Country Schoolmaster and Extension Pioneer; Worthington Landmarks: Photo-Essays of Historic Worthington Properties; and Probing Worthington's Heritage.", group='38')

In [51]:
ds_train.num_labels, ds_valid.num_labels

(13330, 13330)

In [53]:
len(ds_train), len(ds_valid)

(1186239, 306782)